In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen

df_Reports = pd.DataFrame(columns=['Datetime','Sector','Title','Description'])

In [2]:
# RSS Feed for new stuff
# https://www.polizei.sachsen.de/de/presse_rss_pdd.xml


# READ LINKS FROM HISTORY REPORTS
data = pd.read_csv('../data/raw/2018_nov.txt', sep=" ", header=None)
data.columns = ["links"]
data

,links
0,https://www.polizei.sachsen.de/de/60473.htm
1,https://www.polizei.sachsen.de/de/59901.htm
2,https://www.polizei.sachsen.de/de/59218.htm
3,https://www.polizei.sachsen.de/de/58381.htm
4,https://www.polizei.sachsen.de/de/57816.htm
5,https://www.polizei.sachsen.de/de/57211.htm
6,https://www.polizei.sachsen.de/de/56577.htm
7,https://www.polizei.sachsen.de/de/55719.htm
8,https://www.polizei.sachsen.de/de/55335.htm
9,https://www.polizei.sachsen.de/de/54799.htm


In [3]:
link = "https://www.polizei.sachsen.de/de/60473.htm"

f = urlopen(link)
webContent = f.read()
webContent = webContent.decode("utf-8").split("\n")

for line in webContent:
    if("presse" in line and "<a href=" in line): 
        link = line.split("<a href=\"")[1].split("\"")[0]
        print("https://www.polizei.sachsen.de/de/" + link)


https://www.polizei.sachsen.de/de/MI_2017_61023.htm
https://www.polizei.sachsen.de/de/MI_2017_61023.htm
https://www.polizei.sachsen.de/de/MI_2017_61016.htm
https://www.polizei.sachsen.de/de/MI_2017_61016.htm
https://www.polizei.sachsen.de/de/MI_2017_61014.htm
https://www.polizei.sachsen.de/de/MI_2017_61014.htm
https://www.polizei.sachsen.de/de/MI_2017_61004.htm
https://www.polizei.sachsen.de/de/MI_2017_61004.htm
https://www.polizei.sachsen.de/de/MI_2017_60991.htm
https://www.polizei.sachsen.de/de/MI_2017_60991.htm
https://www.polizei.sachsen.de/de/MI_2017_60984.htm
https://www.polizei.sachsen.de/de/MI_2017_60984.htm
https://www.polizei.sachsen.de/de/MI_2017_60970.htm
https://www.polizei.sachsen.de/de/MI_2017_60970.htm
https://www.polizei.sachsen.de/de/MI_2017_60966.htm
https://www.polizei.sachsen.de/de/MI_2017_60966.htm
https://www.polizei.sachsen.de/de/MI_2017_60957.htm
https://www.polizei.sachsen.de/de/MI_2017_60957.htm
https://www.polizei.sachsen.de/de/MI_2017_60947.htm
https://www.

In [4]:
reportPage = "https://www.polizei.sachsen.de/de/MI_2017_60970.htm"

f = urlopen(reportPage)
webContent = f.read()
webContent = webContent.decode("utf-8").split("\n")

datetime = ""
sector = ""
title = ""
descr = ""

def convertChars(pInput):
    pInput = pInput.replace("&auml;","ä");
    pInput = pInput.replace("&Auml;","ä");
    pInput = pInput.replace("&szlig;","ß");
    pInput = pInput.replace("&Ouml;","Ö");
    pInput = pInput.replace("&ouml;","ö");
    pInput = pInput.replace("&Uuml;","Ü");
    pInput = pInput.replace("&uuml;","ü");
     
    return pInput;

for i in range(len(webContent)):
    if("</strong><br />" in webContent[i] and "</u></strong><br />" not in webContent[i]): 
        
        # PARSE TITLE
        title = webContent[i].split("</strong><br />")[0]
        title = title.replace("<strong>", "")
        title = title.strip()
        title = convertChars(title)
        
            
        # PARSE DATETIME
        datetime = webContent[i+2].split("<br />")[0]
        datetime = datetime.replace("Zeit:", "");
        datetime = datetime.replace("&nbsp;", "");
        datetime = datetime.strip()        
        
        # PARSE SECTOR
        sector = webContent[i+3].split("<br />")[0]
        sector = sector.replace("Ort:", "");
        sector = sector.replace("&nbsp;", "");
        sector = sector.strip()
        sector = convertChars(sector)
       
        # PARSE DESCRIPTION
        z = i+4
        maxLenCnt = len(webContent)
        print("nr: {0} = {1} - {2}".format(len(webContent)-i , maxLenCnt, i))
        while(z <= (len(webContent)-i)):
            if("<br />" in webContent[z] and z+1 <= maxLenCnt):
                print("HEAD: " + webContent[z])
                print("CNT: " + webContent[z+1])
                if(webContent[z+1].startswith('<')):
                    break;
                else:
                    descr = descr + webContent[z+1]
            if(z+2 <= maxLenCnt and "<br />" in webContent[z+2]):
                z = z+2;
            else:
                break;
        descr = convertChars(descr)
        #descr = descr.replace("<br />", "\n");
        df_Reports.loc[df_Reports.shape[0]] = [datetime, sector, title, descr]
        descr = "";
        

nr: 286 = 501 - 215
HEAD: <br />
CNT: Gestern Abend haben Dresdner Polizeibeamte einen Jugendlichen (16) festgenommen, der versucht hatte einen Kiosk auf dem Gel&auml;nde der UNI-Klinik an der Fetscherstra&szlig;e zu &uuml;berfallen.<br />
HEAD: <br />
CNT: Kurz vor Ladenschluss betrat der Jugendliche den Kiosk und kaufte ein. An der Kasse zog er pl&ouml;tzlich eine Pistole und forderte von 53-j&auml;hrigen Verk&auml;uferin Bargeld. Die Frau weigerte sich, so dass er hinter den Verkaufstresen kam und sich selbst aus der Kasse bedienen wollte. Dabei kam es zu einer k&ouml;rperlichen Auseinandersetzung, wobei sich die Verk&auml;uferin leicht verletzte. Ein hinzukommender Zeuge half der Frau und hielt den Jugendlichen bis zum Eintreffen der Polizei fest.<br />
HEAD: <br />
CNT: Bei dem Tatverd&auml;chtigen handelt sich um einen 16-j&auml;hrigen Dresdner. Die sichergestellte Pistole entpuppte sich als Gasdruckwaffe ohne Gaszylinder. Die Kriminalpolizei hat die Ermittlungen aufgenommen. (hs

In [5]:

# WE GOT PROBLEMS WITH THE DESCRIPTION PARSING
# Trace: 
# * All descriptions end with <br />,  could be an inidicator about whats wrong
df_Reports

,Datetime,Sector,Title,Description
0,"26.11.2018, 17.55 Uhr",Dresden-Johannstadt,Räuberische Erpressung - Tatverdächtiger gestellt,Gestern Abend haben Dresdner Polizeibeamte ein...
1,"24.11.2018, 14.00 Uhr bis 26.11.2018, 08.55 Uhr",Dresden-äußere Neustadt,Golf aufgebrochen,Am vergangenen Wochenende sind Unbekannte auf ...
2,"23.11.2018, 17.30 Uhr bis 26.11.2018, 07.00 Uhr",Dresden-Prohlis,Einbruch in Arztpraxis,Über das vergangene Wochenende wurde in eine Z...
3,"25.11.2018, 11.00 Uhr bis 26.11.2018, 10.30 Uhr",Stauchitz,Einbruch in zwei Kleintransporter,In der Nacht zum Montag haben Unbekannte auf d...
4,"23.11.2018, 13.00 Uhr bis 26.11.2018, 07.00 Uhr",Großenhain,Einbruch in eine Arztpraxis,
5,"23.11.2018, 18.00 Uhr bis 26.11.2018, 09.30 Uhr",Heidenau,Einbruch in Geschäft,
6,"23.11.2018, 16.00 Uhr bis 26.11.2018, 06.30 Uhr",Sebnitz,Einbruch in Kleintransporter,
